In [3]:
!pip install xgboost
!pip install imbalanced-learn
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn import preprocessing

In [4]:
train=pd.read_csv("pre_processed_train.csv")
test=pd.read_csv("pre_processed_test.csv")

In [5]:
ytrain=train["isFraud"]
train.drop(axis = 1, labels = ["isFraud","Unnamed: 0"], inplace = True)

In [6]:
test.drop(axis = 1, labels = ["Unnamed: 0"], inplace = True)

In [7]:
over = SMOTE(sampling_strategy=0.05)
under = RandomUnderSampler(sampling_strategy=0.8)
steps = [('o', over),('u',under)]
pipeline = Pipeline(steps=steps)
train1, ytrain1 = pipeline.fit_resample(train, ytrain)

In [8]:
from sklearn.preprocessing import StandardScaler
scale= StandardScaler()
X_train = scale.fit_transform(train1)
X_test=scale.transform(test) 

In [9]:
abc = XGBClassifier(
        colsample_bytree= 0.75,
        gamma= 0.65, 
        learning_rate= 0.1,
        max_depth= 20,
        reg_alpha= 0.4,  
        objective="binary:logistic",
        n_estimators=8000,
        njobs=-1
    )       
abc.fit(X_train,ytrain1)

[09:15:24] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "njobs" } are not used.



XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.75,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.65, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=20, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=8000, n_jobs=0, njobs=-1,
              num_parallel_tree=1, predictor='auto', ...)

In [10]:
y_pred1 = abc.predict_proba(X_test)
y_pred1.sum()

147634.98

In [11]:
y_pred = np.array([])
y_pred = []
for i in range(len(y_pred1)):
  if(y_pred1[i][0]<0.35):
    y_pred.append(1)
  else:
    y_pred.append(0)

In [12]:
sum(y_pred)

10376

In [13]:
import csv
with open('xgboost_finalsub.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Id","isFraud"])
    idx=0
    for i in y_pred:
        writer.writerow([idx,i])
        idx=idx+1